In [1]:
import sys
sys.path.append('../../bluesky')  # Adjust the path as necessary

# Now you can import bluesky modules

In [2]:
import bluesky
from bluesky.network.client import Client
import os


from bluesky.network.client import Client



from dotenv import load_dotenv, find_dotenv


import sys
import time



from contextlib import contextmanager


from io import StringIO


from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

Using Python-based geo functions


In [3]:
# Load the .env file
load_dotenv(find_dotenv())

True

In [4]:
# capture output information from the bluesky client and return it as a string
@contextmanager
def capture_stdout():
    new_stdout = StringIO()
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield new_stdout
    finally:
        sys.stdout = old_stdout


def update_until_complete(client):
    complete_output = ""
    while True:
        with capture_stdout() as captured:
            client.update()
        new_output = captured.getvalue()
        if not new_output.strip():
            # No more new output, break the loop
            break
        complete_output += new_output
    client.update()
    return complete_output

In [5]:
# connect to client
client = Client()
client.connect("127.0.0.1", 11000, 11001)
client.update()

Client b'\x00\x1c\x81\x97V' connected to host b'\x00\xd9\x9c!\xdf'
Client active node changed.


In [6]:
client.update()

In [7]:
# # example commands
# command = 'CRE BZB994 A320 -15.101549,-14.504519 48 FL150 150'
# command = 'GETACIDS'
# #command = 'PAN BZB994'
# # sending command to the client. can be a plugin command or a pre-defined command
# client.send_event(b'STACK', command)

In [8]:
# sim_output = update_until_complete(client)
# print("All captured output:\n", sim_output)
# print('-------------------')

In [9]:
client.update()

In [10]:
# initialize the GPT-3.5 model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [11]:

@tool
def get_all_aircraft_info(command: str) -> str:
    """Returns each aircraft information. Use command GETACIDS to get all aircrafts."""
    client.send_event(b'STACK', command)
    sim_output = update_until_complete(client)
    return sim_output


In [65]:
@tool
def send_command_to_simulator(command: str, args: list = None) -> str:

    """Sends a command with optional arguments to the simulator and returns the output.
    Parameters:
    - command (str): The command to send to the simulator.

    - args (list, optional): A list of arguments to include with the command. Defaults to None.

    Returns:

    str: The output from the simulator.
    """
    

    # Ensure args is a list if not provided

    if args is None:

        args = []
        
    else:
        args = [str(item) for item in args]



    # Convert the command and its arguments into a byte string to be sent

    # Note: Assuming that arguments are separated by spaces in the command string

    command_with_args = ' '.join([command] + args)


    # Send the command to the simulator
    #client.update()
    client.send_event(b'STACK', command_with_args)
    
    # wait 1 second
    time.sleep(1)

    # Wait for and retrieve the output from the simulator

    sim_output = update_until_complete(client)

    return sim_output

In [66]:

tools = [send_command_to_simulator]

In [67]:
# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, that can also use tools to answer questions",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [68]:
llm_with_tools = llm.bind_tools(tools)

In [69]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [70]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [119]:

input1 = "Create an airraft for me. it should be a A320 with the A10 callsign, at -15.101549,-14.504519, at 48 heading, flying with the speed of 150 at flight level150."

input2 = "Hey agent, please move the A10 aircraft to the position of longitude -51.604803 and latitude -40.405723"

input3 = "Please change heading of A10 aircraft to 180 degrees."

input4 = "Delete A10 aircraft. I hate this aircraft and don't want to see it anymore."


# Vector database integration

In [72]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import chromadb.utils.embedding_functions as embedding_functions

In [73]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-large"
)

In [74]:
vectordb_path = 'C:/Users/justa/OneDrive/Desktop/Developer/LLM-Enhanced-ATM/llm/skills-library/vectordb'
chroma_client = chromadb.PersistentClient(path=vectordb_path)


In [75]:
# Get a collection object from an existing collection, by name. If it doesn't exist, create it.
collection = chroma_client.get_or_create_collection(
    name="test1", embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})

In [76]:
collection.peek()

{'ids': ['create_aircraft', 'delete_objects', 'move_aircraft', 'set_heading'],
 'embeddings': [[-0.015827570110559464,
   -0.008421882055699825,
   -0.02205001935362816,
   -0.007252563256770372,
   -0.008867337368428707,
   0.03466196358203888,
   -0.00967472419142723,
   0.03744605556130409,
   -0.03711196407675743,
   0.00779546145349741,
   0.03218412026762962,
   -0.019377289339900017,
   -0.006532179191708565,
   -0.030179571360349655,
   -0.0070298356004059315,
   0.0025578858330845833,
   -0.026657693088054657,
   -0.02331678196787834,
   -0.03193354979157448,
   0.004075216595083475,
   -0.00996705424040556,
   0.018388936296105385,
   0.019307687878608704,
   0.037251170724630356,
   0.030569344758987427,
   0.003685443662106991,
   0.029372185468673706,
   0.031376730650663376,
   0.015507400035858154,
   0.005940559320151806,
   0.03666650876402855,
   0.03031877614557743,
   0.017692912369966507,
   -0.03652730584144592,
   0.011386942118406296,
   -0.0006903681205585599,


In [120]:
query_results1 = collection.query(
    query_texts=[input1],
    n_results=3
)

query_results2 = collection.query(
    query_texts=[input2],
    n_results=3
)
query_results3 = collection.query(
    query_texts=[input3],
    n_results=3
)
query_results4 = collection.query(
    query_texts=[input4],
    n_results=3
)

In [121]:
final_input1 = input1 + "\n Below is some useful documentation for you to use in order to generate and send a correct command to the simulator: " + \
    query_results1['documents'][0][0]

In [122]:
final_input2 = input2 + "\n Below is some useful documentation for you to use in order to generate and send a correct command to the simulator: " + \
    query_results2['documents'][0][0]

In [123]:
final_input3 = input3 + "\n Below is some useful documentation for you to use in order to generate and send a correct command to the simulator: " + \
    query_results3['documents'][0][0]

In [124]:
final_input4 = input4 + "\n Below is some useful documentation for you to use in order to generate and send a correct command to the simulator: " + \
    query_results4['documents'][0][0]

In [125]:
out_lst = list(agent_executor.stream({"input": final_input1}))



> Entering new AgentExecutor chain...

Invoking: `send_command_to_simulator` with `{'command': 'CRE', 'args': ['A10', 'A320', -15.101549, -14.504519, 48, 'FL150', 150]}`


The A320 aircraft with the A10 callsign has been successfully created at coordinates -15.101549, -14.504519, flying at a heading of 48, with a speed of 150 at flight level 150.

> Finished chain.


In [126]:
out_lst = list(agent_executor.stream({"input": final_input2}))



> Entering new AgentExecutor chain...

Invoking: `send_command_to_simulator` with `{'command': 'MOVE', 'args': ['A10', -40.405723, -51.604803]}`


The A10 aircraft has been successfully moved to the position with longitude -51.604803 and latitude -40.405723.

> Finished chain.


In [127]:
out_lst = list(agent_executor.stream({"input": final_input3}))




> Entering new AgentExecutor chain...

Invoking: `send_command_to_simulator` with `{'command': 'HDG', 'args': ['A10', '180', 'deg']}`


HDG takes 2 arguments, but 3 were given
Usage:
HDG idx,hdg

Invoking: `send_command_to_simulator` with `{'command': 'HDG', 'args': ['A10', '180']}`
responded: It seems that I provided an extra argument. Let me correct that and send the command again.

The heading of the A10 aircraft has been successfully changed to 180 degrees.

> Finished chain.


In [128]:
out_lst = list(agent_executor.stream({"input": final_input4}))



> Entering new AgentExecutor chain...

Invoking: `send_command_to_simulator` with `{'command': 'DEL', 'args': ['A10']}`


The A10 aircraft has been successfully deleted from the simulation.

> Finished chain.


In [ ]:
# TODO
# 1. connect to a memory block so it will be able to remember the previous commands and responses
# 2. create a vector database with commands and their descriptions (just like voyager did). Input is a prompt, output is top 5 commands
# 3. Make a system that makes the agent write its own commands and test out the commands



# Memory Part (not finished)

In [48]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [77]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage

chat_history = []

In [72]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [111]:
chat_history = []
input1 = "how many aircraft are there in the sky currently?"

result1 = agent_executor.invoke(

    {"input": input1, "chat_history": chat_history})

# chat history should have human message, output result from sim, and output result from openai



> Entering new AgentExecutor chain...

Invoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`


KL102
Aircraft ids: ['KL101', 'KL102', 'KL103', 'BBB999']
Info on KL101 B74F index = 0
Pos: N49'04'7.0"  E006'00'2.0"
Hdg: 180   Trk: 180
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805

Info on KL102 B74F index = 1
Pos: N49'21'14.0"  E004'21'9.0"
Hdg: 210   Trk: 210
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805

Info on KL103 B74F index = 2
Pos: N50'06'37.0"  E003'08'47.0"
Hdg: 239   Trk: 239
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805

Info on BBB999 A320 index = 3
Pos: S14'26'56.0"  W013'45'17.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

There are currently 4 aircraft in the sky. The aircraft IDs are: KL101, KL102, KL103, and BBB999.

> Finished chain.


In [112]:
intermediate_steps1 = result1['intermediate_steps'][0][1]

chat_history.extend(
    [

        HumanMessage(content=input1),
        AIMessage(content=result1["output"]),
        AIMessage(content=intermediate_steps1)
    ]

)
# order here matters for some reason. If the order is changed, for the next question the tool will be invoke even though it is not needed

In [113]:
chat_history

[HumanMessage(content='how many aircraft are there in the sky currently?'),
 AIMessage(content='There are currently 4 aircraft in the sky. The aircraft IDs are: KL101, KL102, KL103, and BBB999.'),
 AIMessage(content='KL102\nAircraft ids: [\'KL101\', \'KL102\', \'KL103\', \'BBB999\']\nInfo on KL101 B74F index = 0\nPos: N49\'04\'7.0"  E006\'00\'2.0"\nHdg: 180   Trk: 180\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL102 B74F index = 1\nPos: N49\'21\'14.0"  E004\'21\'9.0"\nHdg: 210   Trk: 210\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL103 B74F index = 2\nPos: N50\'06\'37.0"  E003\'08\'47.0"\nHdg: 239   Trk: 239\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on BBB999 A320 index = 3\nPos: S14\'26\'56.0"  W013\'45\'17.0"\nHdg: 048   Trk: 048\nAlt: 15000 ft  V/S: 0 fpm\nCAS/TAS/GS: 150/188/188 kts   M: 0.300\n\n')]

In [118]:
input2 = "and which aircraft has the lowest altitude?"
result2 = agent_executor.invoke({"input": input2,
                      "chat_history": chat_history})



> Entering new AgentExecutor chain...


The aircraft with the lowest altitude is BBB999. It is currently at an altitude of 15000 ft.

> Finished chain.


In [121]:
try:
    intermediate_steps2 = result2['intermediate_steps'][0][1]
except IndexError:
    intermediate_steps2 = ''



In [122]:

chat_history.extend(
    [

        HumanMessage(content=input2),
        AIMessage(content=result2["output"]),
        AIMessage(content=intermediate_steps2)
    ]

)
# order here matters for some reason. If the order is changed, for the next question the tool will be invoke even though it is not needed

In [123]:
chat_history

[HumanMessage(content='how many aircraft are there in the sky currently?'),
 AIMessage(content='There are currently 4 aircraft in the sky. The aircraft IDs are: KL101, KL102, KL103, and BBB999.'),
 AIMessage(content='KL102\nAircraft ids: [\'KL101\', \'KL102\', \'KL103\', \'BBB999\']\nInfo on KL101 B74F index = 0\nPos: N49\'04\'7.0"  E006\'00\'2.0"\nHdg: 180   Trk: 180\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL102 B74F index = 1\nPos: N49\'21\'14.0"  E004\'21\'9.0"\nHdg: 210   Trk: 210\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL103 B74F index = 2\nPos: N50\'06\'37.0"  E003\'08\'47.0"\nHdg: 239   Trk: 239\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on BBB999 A320 index = 3\nPos: S14\'26\'56.0"  W013\'45\'17.0"\nHdg: 048   Trk: 048\nAlt: 15000 ft  V/S: 0 fpm\nCAS/TAS/GS: 150/188/188 kts   M: 0.300\n\n'),
 HumanMessage(content='and which aircraft has the lowest altitude?'),
 AIMessage(

In [125]:
input3 = "What is the aircraft id with the lowest altitude?"
result3 = agent_executor.invoke({"input": input3,
                                 "chat_history": chat_history})



> Entering new AgentExecutor chain...


The aircraft ID with the lowest altitude is BBB999.

> Finished chain.


In [126]:
input4 = "What is the alititude of BBB999?"
result4 = agent_executor.invoke({"input": input4,
                                 "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`


The altitude of aircraft BBB999 is 15000 ft.

> Finished chain.


In [128]:
result4['intermediate_steps'][0][1]

''